In [8]:
"""git config --global user.name "gladsd"
git config --global user.email "wlsrudqo14@naver.com"
git remote add origin https://github.com/gladsd/gladsd.git
git add wetie 과제2.ipynb"""

'git config --global user.name "gladsd"\ngit config --global user.email "wlsrudqo14@naver.com"\ngit remote add origin https://github.com/gladsd/gladsd.git\ngit add wetie 과제2.ipynb'

In [4]:


import numpy as np  # 행렬연산
import pandas as pd  # 데이터 처리
import torch  # 파이토치
import torch.nn as nn  # 파이토치 모듈
import torch.nn.init as init  # 초기화 관련 모듈
import torch.optim as optim  # 최적화함수
from torch.utils.data import DataLoader  # 데이터셋을 학습에 용이하게 바꿈
import torch.nn.functional as F  # 자주 쓰는 함수
from sklearn.model_selection import train_test_split  # 데이터 분할
from sklearn.metrics import mean_squared_error, r2_score  # 성능 측정

batch_size = 32
learning_rate = 0.01
num_epoch = 40

# CSV 파일 경로 설정 (여기에 파일 경로를 입력하세요)
file_path = "C:/Users/wlsru/Desktop/wetie_dl/log.csv"

# CSV 파일 로드
try:
    data = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"오류: {file_path} 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    exit()

# 독립 변수와 종속 변수 분리 (종속 변수 열 이름을 'score'로 가정)
try:
    X = data.drop("score", axis=1)  # 종속변수 'score' 열을 제거
    y = data["score"]  # 종속변수 'score' 열을 선택
except KeyError:
    print("오류: 'score' 열을 찾을 수 없습니다. 종속 변수 열 이름을 확인해주세요.")
    exit()

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# PyTorch 텐서로 변환
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# TensorDataset과 DataLoader 생성
train_data = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
test_data = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)


# MLP 모델 정의
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        input_dim = X_train.shape[1]  # X_train의 특성 수 (열 개수에 맞춰 설정)
        
        self.fc1 = nn.Linear(input_dim, 10)  # 첫 번째 레이어
        self.fc2 = nn.Linear(10, 14)  # 두 번째 레이어
        self.fc3 = nn.Linear(14, 100)  # 세 번째 레이어
        
        self.dropout = nn.Dropout(0.5)  # 드롭아웃 추가
        self.batch_norm1 = nn.BatchNorm1d(10)  # 배치 정규화 (첫 번째 레이어)
        self.batch_norm2 = nn.BatchNorm1d(14)  # 배치 정규화 (두 번째 레이어)

    def forward(self, x):
        # 첫 번째 레이어
        x = self.fc1(x)
        x = self.batch_norm1(x)
        x = F.relu(x)  # 활성화 함수
        x = self.dropout(x)  # 드롭아웃

        # 두 번째 레이어
        x = self.fc2(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = self.dropout(x)

        # 세 번째 레이어
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)  # softmax 출력
        return x

    def weight_init(m):
        if isinstance(m, nn.Linear):
            init.kaiming_uniform_(m.weight.data)  # He 초기화

# 모델 인스턴스 생성
model = MLP()
model.apply(MLP.weight_init)

# Optimizer와 Loss Function 정의
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

# 학습 함수
def train(model, train_loader, optimizer):
    model.train()  # 모델을 학습 모드로 설정
    for batch_idx, (image, label) in enumerate(train_loader):  # 배치 단위로 학습
        optimizer.zero_grad()  # 기울기 초기화
        output = model(image)  # 모델 예측값
        loss = loss_fn(output, label)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 가중치 업데이트

# 평가 함수
def evaluate(model, test_loader):
    model.eval()  # 모델을 평가 모드로 설정
    test_loss = 0
    correct = 0

    with torch.no_grad():  # 평가 시에는 기울기를 계산하지 않음
        for image, label in test_loader:
            output = model(image)
            test_loss += loss_fn(output, label).item()  # 손실 누적
            prediction = output.max(1, keepdim=True)[1]  # 가장 높은 확률을 가진 클래스 예측
            correct += prediction.eq(label.view_as(prediction)).sum().item()  # 정확도 계산
    
    test_loss /= len(test_loader.dataset)  # 평균 손실
    test_accuracy = 100. * correct / len(test_loader.dataset)  # 정확도 계산
    return test_loss, test_accuracy

# 학습 및 평가
epoch_list = []
test_loss_list = []
test_accuracy_list = []

for epoch in range(1, num_epoch + 1):
    train(model, train_loader, optimizer)  # 모델 학습
    test_loss, test_accuracy = evaluate(model, test_loader)  # 모델 평가
    
    epoch_list.append(epoch)
    test_loss_list.append(test_loss)
    test_accuracy_list.append(test_accuracy)
    
    print("[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} %".format(
        epoch, test_loss, test_accuracy
    ))

[EPOCH: 1], 	Test Loss: 0.0784, 	Test Accuracy: 14.90 %
[EPOCH: 2], 	Test Loss: 0.0743, 	Test Accuracy: 16.85 %
[EPOCH: 3], 	Test Loss: 0.0721, 	Test Accuracy: 17.55 %
[EPOCH: 4], 	Test Loss: 0.0712, 	Test Accuracy: 17.50 %


KeyboardInterrupt: 